In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 https://cloud.r-project.org/bin/li

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-07-17 20:50:30--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-07-17 20:50:30 (6.49 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33605939| RGYFDX8QXKEIR|B007KO2MLO|     328837464|           Big Maria|Digital_Ebook_Pur...|          4|            0|          0|   N|                N|              Quirky|Elmore Leonard me...| 2013-09-09|
|         US|   34058393|R13CBGTMNV9R8Z|B005FLODDE|     764276359|The Woman Who Was...|Digital_Ebook_Pur

In [13]:
from pyspark.sql.functions import to_date
from pyspark.sql.functions import col
# Read in the Review dataset as a DataFrame

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RGYFDX8QXKEIR|          4|            0|          0|   N|                N|
|R13CBGTMNV9R8Z|          4|            1|          2|   N|                Y|
| R7DRFHC0F71O0|          5|            0|          0|   N|                N|
|R27LUKEXU3KBXQ|          5|            1|          1|   N|                Y|
|R1VXTPUYMNU687|          5|            1|          2|   N|                N|
|R30DKW1GJWLPZC|          3|            1|          2|   N|                Y|
|R18DPFG2FALJI9|          5|            0|          0|   N|                Y|
|R24D677N5WBW5Q|          5|            0|          0|   N|                Y|
|R2FCJ9BQLSIOR3|          5|            0|          0|   N|                Y|
|R1R6K4MAKDWTXI|          4|            0|          0|   N|     

In [8]:
#filter by total voteshelpful vote count 20 or greater 
total_filter_df = vine_df[vine_df['total_votes'] >= 20]
total_filter_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R39ESX43X1SA5T|          1|           36|         47|   N|                Y|
|R31QRZ3JAMPBB4|          1|           16|         26|   N|                Y|
|R23FKBEXURC3SN|          5|           28|         36|   N|                N|
| R4IAIDV5EE84W|          1|            8|         21|   N|                Y|
|R26MB2DA7ROT4P|          1|            2|         21|   N|                Y|
| RTRVYV0GWUTCK|          5|           18|         25|   N|                Y|
|R3QLZAI37SNOOC|          2|           30|         31|   N|                Y|
|R1QYZU3RS4CY4F|          5|           66|         76|   N|                N|
|R2UR5OLN1BU87J|          2|           21|         23|   N|                Y|
| RYBZ6CTBOGI0H|          1|            3|         84|   N|     

In [14]:
#helpful votes diveded by total votes equal or greater than 50%
helpful_filter_df = total_filter_df.filter(col("helpful_votes")/col("total_votes") >= .50)
helpful_filter_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R39ESX43X1SA5T|          1|           36|         47|   N|                Y|
|R31QRZ3JAMPBB4|          1|           16|         26|   N|                Y|
|R23FKBEXURC3SN|          5|           28|         36|   N|                N|
| RTRVYV0GWUTCK|          5|           18|         25|   N|                Y|
|R3QLZAI37SNOOC|          2|           30|         31|   N|                Y|
|R1QYZU3RS4CY4F|          5|           66|         76|   N|                N|
|R2UR5OLN1BU87J|          2|           21|         23|   N|                Y|
|R38NW44NY19VAZ|          5|           30|         32|   N|                Y|
| RLUFKZ223OR3L|          2|           20|         23|   N|                N|
|R3H9D0U7BCE6FU|          4|           23|         24|   N|     

In [19]:
#filter data to show reviews that were written as part of a PAID vine program 
#no reviews where found as part of the vine program 
paid_vine_filter_df = helpful_filter_df.filter(col("vine") == 'Y')
paid_vine_filter_df.show()

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [20]:
paid_vine_filter_df.count()

0

In [27]:
#filter data where a review was not apart of the vine program - unpaid reviews 
unpaid_vine_filter_df = helpful_filter_df.filter(col("vine") == 'N')
unpaid_vine_filter_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R39ESX43X1SA5T|          1|           36|         47|   N|                Y|
|R31QRZ3JAMPBB4|          1|           16|         26|   N|                Y|
|R23FKBEXURC3SN|          5|           28|         36|   N|                N|
| RTRVYV0GWUTCK|          5|           18|         25|   N|                Y|
|R3QLZAI37SNOOC|          2|           30|         31|   N|                Y|
|R1QYZU3RS4CY4F|          5|           66|         76|   N|                N|
|R2UR5OLN1BU87J|          2|           21|         23|   N|                Y|
|R38NW44NY19VAZ|          5|           30|         32|   N|                Y|
| RLUFKZ223OR3L|          2|           20|         23|   N|                N|
|R3H9D0U7BCE6FU|          4|           23|         24|   N|     

In [21]:
#total number of reviews 
df.count()

5101693

In [25]:
# number of 5-star reviews 
five_star_rating_df = vine_df.filter(col("star_rating") == 5)
five_star_rating_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R7DRFHC0F71O0|          5|            0|          0|   N|                N|
|R27LUKEXU3KBXQ|          5|            1|          1|   N|                Y|
|R1VXTPUYMNU687|          5|            1|          2|   N|                N|
|R18DPFG2FALJI9|          5|            0|          0|   N|                Y|
|R24D677N5WBW5Q|          5|            0|          0|   N|                Y|
|R2FCJ9BQLSIOR3|          5|            0|          0|   N|                Y|
|R3R5DILCWM8J7B|          5|            0|          0|   N|                Y|
|R3K9PJU5GLDY3O|          5|            1|          2|   N|                Y|
|R1KTZMCDOJXAEK|          5|            0|          0|   N|                N|
|R3SBEH4Y3W9W11|          5|            0|          0|   N|     

In [26]:
#% of 5-star reviews for the twop types of reviews (paid vs unpaid)
paid_vine_filter2_df = five_star_rating_df.filter(col("vine") == 'Y')
paid_vine_filter2_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3E4BDOHWZTN9V|          5|            0|          0|   Y|                N|
|R36PP7VFA44WYN|          5|            1|          1|   Y|                N|
| RFTD5UXIR6GN2|          5|            0|          0|   Y|                N|
| RNHN23JW3QYI0|          5|            1|          1|   Y|                N|
|R2PFGNIY6271GN|          5|            1|          1|   Y|                Y|
|R36RPLF7MN74TX|          5|            2|          4|   Y|                Y|
|R1PT1PA2A49IM4|          5|            0|          0|   Y|                Y|
| R8R13247MPATI|          5|            2|          5|   Y|                N|
|R326NZV03DVAZ8|          5|            7|          9|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----

In [29]:
unpaid_vine_filter2_df = five_star_rating_df.filter(col("vine") == 'N')
unpaid_vine_filter2_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R7DRFHC0F71O0|          5|            0|          0|   N|                N|
|R27LUKEXU3KBXQ|          5|            1|          1|   N|                Y|
|R1VXTPUYMNU687|          5|            1|          2|   N|                N|
|R18DPFG2FALJI9|          5|            0|          0|   N|                Y|
|R24D677N5WBW5Q|          5|            0|          0|   N|                Y|
|R2FCJ9BQLSIOR3|          5|            0|          0|   N|                Y|
|R3R5DILCWM8J7B|          5|            0|          0|   N|                Y|
|R3K9PJU5GLDY3O|          5|            1|          2|   N|                Y|
|R1KTZMCDOJXAEK|          5|            0|          0|   N|                N|
|R3SBEH4Y3W9W11|          5|            0|          0|   N|     

In [32]:
paid_vine_filter2_df.count()

9

In [33]:
unpaid_vine_filter2_df.count()

2952604

In [34]:
paid_vs_unpaid = (paid_vine_filter2_df.count()/unpaid_vine_filter2_df.count())*100
paid_vs_unpaid

0.000304815681344332